## Use Full Auth to handle Domo API Authentication

- "full authentication" is not the same as client_id and secret authentication which can be retrieved from developer.domo.com

- client_id and secret auth (we will call developer_token authentication moving forward) only applies to a small handful of APIs documented under developer.domo.com

- "full authentication actually has two flavors.<br>
    - username and password authentication (session_token which has a short expiration date)
    - access_token authentication (Admin > Security > Access Token) which tends to have a longer expiration date, we will cover this later


DEVELOPER TIPS:
- document the type we expect to receive for each input and output variable (str)
- provide a document string (denoted in triple quotes) to describe what the function does
- notice the use of logic to test for success

- 🆕notebooks allow you to compartmentalize your code, write a simple test under each function

In [1]:
import domojupyter as domo
import json

def get_account_credentials(account_name, is_abstract_account: bool = True):
    """handles retrieving account properties in domo jupter"""
    
    account_properties = domo.get_account_property_keys(account_name)
    res = {prop: domo.get_account_property_value(account_name, prop) for prop in account_properties}
    return json.loads(res['credentials'])

creds = get_account_credentials("username_password_auth", is_abstract_account=True)
assert creds

DEVELOPER TIP:
create custom Exception classes to handle error catching in your application

when you initialize the `DomoAPIRequest_Error` class, you must pass an error `message`.  
that message gets passed to the parent function `Exception` via the `super().__init__()` call

In [20]:
class DomoAPIRequest_Error(Exception):
    """a customized Exception class for handing Domo errors"""
    
    def __init__(self, message):
        super().__init__(message)

In [21]:
import requests

def get_full_auth(
    domo_instance: str,
    domo_username: str,
    domo_password: str
) -> str:  # returns a session token
    """use username and password to generate an access token"""

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    res = requests.request(method="POST", url=url, json=body)
    data = res.json()

    token = data.get("sessionToken")
    
    if not token:
        raise DomoAPIRequest_Error("unable to retrieve a session token")

    return token

session_token = get_full_auth(domo_username = creds['DOMO_USERNAME'],
                              domo_password = creds['DOMO_PASSWORD'],
                              domo_instance = creds['DOMO_INSTANCE'] )
assert session_token

DEVELOPER TIPS
you could have two functions, one for handling any account type, and one specifically for handling abstract credentials.

in the case of abstract credentials, we only want the "credentials" key from the JSON created (notice the modified return statement)

In the v2 of this function, we'll pass `is_abstract_account` boolean parameter with a default value of `True` to modify how `get_account_credentials` behaves

In [3]:
import json

# to accommod
def get_account_credentials(account_name, is_abstract_account: bool = True):
    """version 2 of get_account credentaials"""
    
    account_properties = domo.get_account_property_keys(account_name)
    res = {prop: domo.get_account_property_value(account_name, prop) for prop in account_properties}
    return json.loads(res['credentials'])

creds = get_account_credentials("username_password_auth")
# creds

In [22]:
def get_accounts(domo_instance, session_token):
    """retrieves a list of accounts this user has access to"""
    
    headers = {"x-domo-authentication": session_token}    
    url = f'https://{domo_instance}.domo.com/api/data/v2/datasources/providers'
    
    res= requests.request(method = "GET",
                            url = url,
                            headers = headers
                           )
    
    data = res.json()
    
    if not res.ok:
        DomoAPIRequestError(data)
        
    return data

In [24]:

def main():
    
    creds = get_account_credentials('username_password_auth')

    session_token = get_full_auth(domo_username = creds['DOMO_USERNAME'],
                                  domo_password = creds['DOMO_PASSWORD'],
                                  domo_instance = creds['DOMO_INSTANCE']
                                )

    return get_accounts(domo_instance = domo_instance, session_token = session_token)

main()

[{'key': 'abstract-credential-store',
  'name': 'Abstract Credential Store',
  'authenticationScheme': 'fields',
  'unassociatedDataSourceCount': 0,
  'accounts': [{'id': 71,
    'name': 'domo_creds',
    'userId': '1893952720',
    'displayName': 'domo_creds - update 2024-02-21',
    'type': 'data',
    'dataProviderType': 'abstract-credential-store',
    'valid': True,
    'dateOfExpiration': None,
    'dataSourceCount': 0,
    'daysToExpiry': None,
    'expired': None},
   {'id': 87,
    'name': 'Abstract Credential Store Account',
    'userId': '1893952720',
    'displayName': 'jw_creds',
    'type': 'data',
    'dataProviderType': 'abstract-credential-store',
    'valid': True,
    'dateOfExpiration': None,
    'dataSourceCount': 0,
    'daysToExpiry': None,
    'expired': None},
   {'id': 88,
    'name': 'fake_account',
    'userId': '1893952720',
    'displayName': 'fake_account',
    'type': 'data',
    'dataProviderType': 'abstract-credential-store',
    'valid': True,
    'da